In [4]:
import garminconnect
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
load_dotenv()

True

In [5]:
email = os.getenv("email")
password = os.getenv("garminpassword")\

garmin = garminconnect.Garmin(email, password)
garmin.login()

garmin.display_name

'5f3dc0bc-4916-4755-894b-cca99162502b'

**Export sleep movment data to timeseries table** 

In [29]:
import json
date = '2025-06-12'

data = garmin.get_sleep_data(date)
general_sleep_data = data['dailySleepDTO']
df_general = pd.DataFrame(general_sleep_data)

print(df_general.head())

                          id  userProfilePK calendarDate  sleepTimeSeconds  \
totalDuration  1749687960000       80897268   2025-06-12             24660   
stress         1749687960000       80897268   2025-06-12             24660   
awakeCount     1749687960000       80897268   2025-06-12             24660   
overall        1749687960000       80897268   2025-06-12             24660   
remPercentage  1749687960000       80897268   2025-06-12             24660   

               napTimeSeconds  sleepWindowConfirmed  \
totalDuration               0                  True   
stress                      0                  True   
awakeCount                  0                  True   
overall                     0                  True   
remPercentage               0                  True   

              sleepWindowConfirmationType  sleepStartTimestampGMT  \
totalDuration    enhanced_confirmed_final           1749687960000   
stress           enhanced_confirmed_final           1749687960

In [30]:
sleep_movement_data = data['sleepMovement']
df_sleep_movement = pd.DataFrame(sleep_movement_data)
df_sleep_movement['startGMT'] = pd.to_datetime(df_sleep_movement['startGMT'])
df_sleep_movement['endGMT'] = pd.to_datetime(df_sleep_movement['endGMT'])
df_sleep_movement['calendarDate'] = date

print(df_sleep_movement.head())

             startGMT              endGMT  activityLevel calendarDate
0 2025-06-11 23:26:00 2025-06-11 23:27:00       5.520759   2025-06-12
1 2025-06-11 23:27:00 2025-06-11 23:28:00       5.036847   2025-06-12
2 2025-06-11 23:28:00 2025-06-11 23:29:00       4.529466   2025-06-12
3 2025-06-11 23:29:00 2025-06-11 23:30:00       4.008012   2025-06-12
4 2025-06-11 23:30:00 2025-06-11 23:31:00       3.482673   2025-06-12


**spo2 Data**

In [7]:
spo2_data = data['wellnessEpochSPO2DataDTOList']
df_spo2 = pd.DataFrame(spo2_data)
df_spo2['epochTimestamp'] = pd.to_datetime(df_spo2['epochTimestamp'])

df_spo2['calendarDate'] = pd.to_datetime(df_spo2['calendarDate'])

print(df_spo2.head())



   userProfilePK      epochTimestamp    deviceId calendarDate  epochDuration  \
0       80897268 2025-06-12 00:26:00  3339097678   2025-06-12             60   
1       80897268 2025-06-12 00:27:00  3339097678   2025-06-12             60   
2       80897268 2025-06-12 00:28:00  3339097678   2025-06-12             60   
3       80897268 2025-06-12 00:29:00  3339097678   2025-06-12             60   
4       80897268 2025-06-12 00:30:00  3339097678   2025-06-12             60   

   spo2Reading  readingConfidence  
0           95                 17  
1           96                 14  
2           96                 25  
3           94                 19  
4           94                  6  


In [8]:
respiration_data = data['wellnessEpochRespirationDataDTOList']
df_respiration = pd.DataFrame(respiration_data)
df_respiration['startTimeGMT'] = pd.to_datetime(df_respiration['startTimeGMT'], unit='ms')


print(df_respiration)

           startTimeGMT  respirationValue
0   2025-06-12 00:26:00              13.0
1   2025-06-12 00:26:00              13.0
2   2025-06-12 00:28:00              13.0
3   2025-06-12 00:30:00              13.0
4   2025-06-12 00:32:00              13.0
..                  ...               ...
210 2025-06-12 07:24:00              15.0
211 2025-06-12 07:26:00              15.0
212 2025-06-12 07:28:00              16.0
213 2025-06-12 07:30:00              16.0
214 2025-06-12 07:32:00              16.0

[215 rows x 2 columns]


In [9]:
sleephr_data = data['sleepHeartRate']
df_sleephr = pd.DataFrame(sleephr_data)
df_sleephr['startGMT'] = pd.to_datetime(df_sleephr['startGMT'], unit='ms')


print(df_sleephr)

     value            startGMT
0       82 2025-06-12 00:26:00
1       82 2025-06-12 00:28:00
2       79 2025-06-12 00:30:00
3       76 2025-06-12 00:32:00
4       78 2025-06-12 00:34:00
..     ...                 ...
209     63 2025-06-12 07:24:00
210     62 2025-06-12 07:26:00
211     63 2025-06-12 07:28:00
212     64 2025-06-12 07:30:00
213     64 2025-06-12 07:32:00

[214 rows x 2 columns]


In [10]:
sleep_stress_data = data['sleepStress']
df_sleep_stress = pd.DataFrame(sleep_stress_data)
df_sleep_stress['startGMT'] = pd.to_datetime(df_sleep_stress['startGMT'], unit='ms')


print(df_sleep_stress)

     value            startGMT
0       80 2025-06-12 00:24:00
1       70 2025-06-12 00:27:00
2       52 2025-06-12 00:30:00
3       46 2025-06-12 00:33:00
4       24 2025-06-12 00:36:00
..     ...                 ...
139     21 2025-06-12 07:21:00
140     21 2025-06-12 07:24:00
141     23 2025-06-12 07:27:00
142     25 2025-06-12 07:30:00
143     31 2025-06-12 07:33:00

[144 rows x 2 columns]


In [11]:
sleep_BB_data = data['sleepBodyBattery']
df_sleep_BB = pd.DataFrame(sleep_BB_data)
df_sleep_BB['startGMT'] = pd.to_datetime(df_sleep_BB['startGMT'], unit='ms')


print(df_sleep_BB)

     value            startGMT
0       10 2025-06-12 00:24:00
1       10 2025-06-12 00:27:00
2       10 2025-06-12 00:30:00
3       10 2025-06-12 00:33:00
4       10 2025-06-12 00:36:00
..     ...                 ...
139     75 2025-06-12 07:21:00
140     75 2025-06-12 07:24:00
141     75 2025-06-12 07:27:00
142     75 2025-06-12 07:30:00
143     75 2025-06-12 07:33:00

[144 rows x 2 columns]


In [12]:
sleep_HRV_data = data['hrvData']
df_sleep_HRV = pd.DataFrame(sleep_HRV_data)
df_sleep_HRV['startGMT'] = pd.to_datetime(df_sleep_HRV['startGMT'], unit='ms')


print(df_sleep_HRV)

    value            startGMT
0    22.0 2025-06-12 00:26:14
1    23.0 2025-06-12 00:31:14
2    33.0 2025-06-12 00:36:14
3    42.0 2025-06-12 00:41:14
4    53.0 2025-06-12 00:46:14
..    ...                 ...
81   60.0 2025-06-12 07:11:14
82   57.0 2025-06-12 07:16:14
83   58.0 2025-06-12 07:21:14
84   57.0 2025-06-12 07:26:14
85   53.0 2025-06-12 07:31:14

[86 rows x 2 columns]


In [13]:
sleep_levels_data = data['sleepLevels']
df_sleep_levels = pd.DataFrame(sleep_levels_data)
df_sleep_levels['startGMT'] = pd.to_datetime(df_sleep_levels['startGMT'])
df_sleep_levels['endGMT'] = pd.to_datetime(df_sleep_levels['endGMT'])

print(df_sleep_levels)

              startGMT              endGMT  activityLevel
0  2025-06-12 00:26:00 2025-06-12 00:50:00            1.0
1  2025-06-12 00:50:00 2025-06-12 01:22:00            0.0
2  2025-06-12 01:22:00 2025-06-12 01:35:00            1.0
3  2025-06-12 01:35:00 2025-06-12 01:47:00            0.0
4  2025-06-12 01:47:00 2025-06-12 02:02:00            1.0
5  2025-06-12 02:02:00 2025-06-12 02:27:00            0.0
6  2025-06-12 02:27:00 2025-06-12 02:39:00            1.0
7  2025-06-12 02:39:00 2025-06-12 03:12:00            0.0
8  2025-06-12 03:12:00 2025-06-12 04:03:00            1.0
9  2025-06-12 04:03:00 2025-06-12 04:17:00            0.0
10 2025-06-12 04:17:00 2025-06-12 04:29:00            1.0
11 2025-06-12 04:29:00 2025-06-12 04:44:00            2.0
12 2025-06-12 04:44:00 2025-06-12 05:15:00            1.0
13 2025-06-12 05:15:00 2025-06-12 05:28:00            2.0
14 2025-06-12 05:28:00 2025-06-12 05:55:00            1.0
15 2025-06-12 05:55:00 2025-06-12 06:26:00            2.0
16 2025-06-12 

(              startGMT              endGMT  activityLevel
 0  2025-06-12 00:26:00 2025-06-12 00:50:00            1.0
 1  2025-06-12 00:50:00 2025-06-12 01:22:00            0.0
 2  2025-06-12 01:22:00 2025-06-12 01:35:00            1.0
 3  2025-06-12 01:35:00 2025-06-12 01:47:00            0.0
 4  2025-06-12 01:47:00 2025-06-12 02:02:00            1.0
 5  2025-06-12 02:02:00 2025-06-12 02:27:00            0.0
 6  2025-06-12 02:27:00 2025-06-12 02:39:00            1.0
 7  2025-06-12 02:39:00 2025-06-12 03:12:00            0.0
 8  2025-06-12 03:12:00 2025-06-12 04:03:00            1.0
 9  2025-06-12 04:03:00 2025-06-12 04:17:00            0.0
 10 2025-06-12 04:17:00 2025-06-12 04:29:00            1.0
 11 2025-06-12 04:29:00 2025-06-12 04:44:00            2.0
 12 2025-06-12 04:44:00 2025-06-12 05:15:00            1.0
 13 2025-06-12 05:15:00 2025-06-12 05:28:00            2.0
 14 2025-06-12 05:28:00 2025-06-12 05:55:00            1.0
 15 2025-06-12 05:55:00 2025-06-12 06:26:00            2

In [17]:
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
dbname = os.getenv("dbname")

DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(DATABASE_URL)

try:
    with engine.connect() as conn:
        result = result = conn.execute(text("SELECT version();"))
        for row in result:
            print("✅ Connected to:", row[0])
except Exception as e:
    print("❌ Connection failed:", e)

✅ Connected to: PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-bit


In [22]:
frames = {
    "sleep_general": df_general,
    "sleep_levels": df_sleep_levels,
    "sleep_hrv": df_sleep_HRV,
    "sleep_BB": df_sleep_BB,
    "sleep_stress":df_sleep_stress,
    "sleep_hr": df_sleephr,
    "sleep_respiration": df_respiration,
    "sleep_spo2": df_spo2
}

# Identify problematic columns
for df in frames.values():
    for col in df.columns:
        if df[col].apply(type).eq(dict).any():
            df[col] = df[col].apply(json.dumps)
 
for table_name, df in frames.items():
    df['calendarDate'] = date
    df.to_sql(table_name, con=engine, if_exists="append", index=False)
